<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#housekeeping" data-toc-modified-id="housekeeping-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>housekeeping</a></span><ul class="toc-item"><li><span><a href="#languages-&amp;-packages" data-toc-modified-id="languages-&amp;-packages-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>languages &amp; packages</a></span></li><li><span><a href="#env-set-up" data-toc-modified-id="env-set-up-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>env set-up</a></span></li><li><span><a href="#parameters" data-toc-modified-id="parameters-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>parameters</a></span></li><li><span><a href="#equations" data-toc-modified-id="equations-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>equations</a></span></li></ul></li><li><span><a href="#whole-ODE-system-numeric-solve" data-toc-modified-id="whole-ODE-system-numeric-solve-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>whole ODE system numeric solve</a></span></li><li><span><a href="#$P$-&amp;-partial-$C$-equation-isolation-numeric-solve" data-toc-modified-id="$P$-&amp;-partial-$C$-equation-isolation-numeric-solve-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>$P$ &amp; partial-$C$ equation isolation numeric solve</a></span></li><li><span><a href="#$P$-&amp;-partial-$C$-equation-isolation-numeric-solve" data-toc-modified-id="$P$-&amp;-partial-$C$-equation-isolation-numeric-solve-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>$P$ &amp; partial-$C$ equation isolation numeric solve</a></span></li></ul></div>

# housekeeping
This notebook is for analytically (2nd attempt) solving parameters in the set of ODEs for eco-bioelectric cell (EBC).

## languages & packages
Julia-lang 1.3.1: `PyCall` (1.91.2), `DataFrames` (0.20.0), `SymPy` (1.0.18)

## env set-up

In [1]:
using PyCall, DataFrames, SymPy
sympy.init_printing() ## use other py3 function not implemented in native Julia-lang
println()

In [12]:
C,P,B,K_P, g_P,e_P,R_P,m_P, g_B,e_B,R_B,m_B = symbols("C P B K_P g_P e_P R_P m_P g_B e_B R_B m_B", positive=true)

(C, P, B, K_P, g_P, e_P, R_P, m_P, g_B, e_B, R_B, m_B)

## parameters
symbol | unit | meaning
--- | --- | ---
$C$ | kgCm$^-3$ | system organic __C__arbon pool concentration
$P$ | kgCm$^-3$ | system __P__hytoplankton Carbon pool concentration
$B$ | kgCm$^-3$ | system _E. coli_ (__B__acteria) Carbon pool concentration
$K_P$ | kgCm$^-3$ | system phytoplankton Carbon pool carrying capacity
$g_P$ | t$^{-1}$ | growth rate of $P$
$e_P$ | no | carbon assimilation efficiency of $P$
$R_P$ | t$^{-1}$ | respiration rate of $P$
$m_P$ | t$^{-1}$ | death rate of $P$
$g_B$ | m$^3$(kgC$\cdot$t)$^{-1}$ | growth rate per $B$ pool concentration unit
$e_B$ | no | absorption efficiency of $B$
$R_B$ | t$^{-1}$ | respiration rate of $B$
$m_B$ | t$^{-1}$ | death rate of $B$

## equations
\begin{equation}
\left\{\begin{array}{rl}
C'(t) &= (g_P(1-\varepsilon_P)\left(1-\frac{P}{K_P}\right) + m_P)P + (g_B(1-\varepsilon_B)C + m_B)B\\
P'(t) &= (g_P\varepsilon_P \left(1-\frac{P}{K_P}\right) - R_P - m_P)P\\
B'(t) &= (g_B\varepsilon_BC - R_B - m_B)B\\
\end{array}\right.
\end{equation}

# whole ODE system numeric solve

# $P$ & partial-$C$ equation isolation numeric solve

# $P$ & partial-$C$ equation isolation numeric solve